In [108]:
import json
import pandas as pd
from pymongo import MongoClient
import spacy
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

from pathlib import Path
from dotenv import load_dotenv
import os

In [109]:
# get path to ENV info pass words
env_path = Path('../.env') 
load_dotenv(dotenv_path=env_path)

mongo_url = os.getenv("MONGODB_URI")
db_name = os.getenv("DB_NAME")


In [110]:
# Open server and get data
client = MongoClient(mongo_url)
db = client[db_name]
collection = db['spanish_attempts']

data = list(collection.find())

In [111]:
flattened_mistakes = []

for doc in data:
    timestamp = doc.get('timestamp')
    original_sentence = doc.get('original')
    corrected_sentence = doc.get('corrected')

    for mistake in doc.get('mistakes', []):
        category_value = mistake.get('category', None)
        if isinstance(category_value, list):
            category_value = category_value[0] if category_value else None
        flattened_mistakes.append({
            'original_sentence': original_sentence,
            'corrected_sentence': corrected_sentence,
            'original_mistake': mistake.get('original'),
            'correction': mistake.get('correction'),
            'mistake_type': mistake.get('type'),
            'category': category_value,  # take first category
            'explanation': mistake.get('explanation'),
            'timestamp': timestamp
        })

mistakes_df = pd.DataFrame(flattened_mistakes)
print(f"Total mistakes: {len(mistakes_df)}")
mistakes_df.head()
    

Total mistakes: 48


,original_sentence,corrected_sentence,original_mistake,correction,mistake_type,category,explanation,timestamp
0,Me gusta haciendo proyectos que me encantan. E...,Me gusta hacer proyectos que me encantan. Ento...,haciendo,hacer,error,None,The verb 'gustar' is followed by an infinitive...,2025-09-15 13:35:44.266
1,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,vocabulary,The word 'comentar' is more appropriate here a...,2025-09-15 13:43:26.877
2,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,preposition,The correct preposition to use with 'comentar'...,2025-09-15 13:43:26.877
3,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,vocabulary,"In this context, 'aportar' meaning 'contribute...",2025-09-15 13:43:26.877
4,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,vocabulary,'En ocasiones' meaning 'at times' fits better ...,2025-09-15 13:43:26.877


In [112]:
mistakes_df['original_sentence_cleaned'] = mistakes_df['original_sentence'].str.lower().str.strip()
mistakes_df['corrected_sentence_cleaned'] = mistakes_df['corrected_sentence'].str.lower().str.strip()
mistakes_df['original_mistake_cleaned'] = mistakes_df['original_mistake'].str.lower().str.strip()
mistakes_df['correction_cleaned'] = mistakes_df['correction'].str.lower().str.strip()

category_map = {
    'phrase flow': 'phraseFlow',
    'conjugation': 'conjugation',
    'gender/number agreement': 'genderNumberAgreement',
    'preposition': 'preposition',
    'vocabulary': 'vocabulary',
    'articles': 'articles',
    'pronouns': 'pronouns',
    'idiomatic expressions': 'idiomaticExpressions',
    'punctuation': 'punctuation'
}

mistakes_df['category_standard'] = mistakes_df['category'].map(category_map).fillna(mistakes_df['category'])

mistakes_df.head()

,original_sentence,corrected_sentence,original_mistake,correction,mistake_type,category,explanation,timestamp,original_sentence_cleaned,corrected_sentence_cleaned,original_mistake_cleaned,correction_cleaned,category_standard
0,Me gusta haciendo proyectos que me encantan. E...,Me gusta hacer proyectos que me encantan. Ento...,haciendo,hacer,error,None,The verb 'gustar' is followed by an infinitive...,2025-09-15 13:35:44.266,me gusta haciendo proyectos que me encantan. e...,me gusta hacer proyectos que me encantan. ento...,haciendo,hacer,None
1,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,vocabulary,The word 'comentar' is more appropriate here a...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary
2,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,preposition,The correct preposition to use with 'comentar'...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition
3,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,vocabulary,"In this context, 'aportar' meaning 'contribute...",2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary
4,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,vocabulary,'En ocasiones' meaning 'at times' fits better ...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary


In [113]:
# load spanish nlp model
nlp = spacy.load('es_core_news_sm')

# load similarity model 
model = SentenceTransformer('all-MiniLM-L6-v2')

#Encoding for only the original sentence
embeddings_orig = model.encode(mistakes_df['original_sentence_cleaned'].tolist(), convert_to_tensor=True)
embeddings_corr = model.encode(mistakes_df['corrected_sentence_cleaned'].tolist(), convert_to_tensor=True)

mistakes_df['sentence_similarity'] = [util.cos_sim(o, c).item() for o, c in zip(embeddings_orig, embeddings_corr)]

# Encode only the original mistake and its correction
embeddings_orig = model.encode(mistakes_df['original_mistake_cleaned'].tolist(), convert_to_tensor=True)
embeddings_corr = model.encode(mistakes_df['correction_cleaned'].tolist(), convert_to_tensor=True)

mistakes_df['mistake_similarity'] = [util.cos_sim(o, c).item() for o, c in zip(embeddings_orig, embeddings_corr)]

mistakes_df.head(10)

,original_sentence,corrected_sentence,original_mistake,correction,mistake_type,category,explanation,timestamp,original_sentence_cleaned,corrected_sentence_cleaned,original_mistake_cleaned,correction_cleaned,category_standard,sentence_similarity,mistake_similarity
0,Me gusta haciendo proyectos que me encantan. E...,Me gusta hacer proyectos que me encantan. Ento...,haciendo,hacer,error,None,The verb 'gustar' is followed by an infinitive...,2025-09-15 13:35:44.266,me gusta haciendo proyectos que me encantan. e...,me gusta hacer proyectos que me encantan. ento...,haciendo,hacer,None,0.985953,0.531973
1,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,vocabulary,The word 'comentar' is more appropriate here a...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,0.932277,0.286470
2,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,preposition,The correct preposition to use with 'comentar'...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,0.932277,0.685403
3,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,vocabulary,"In this context, 'aportar' meaning 'contribute...",2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,0.932277,0.348006
4,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,vocabulary,'En ocasiones' meaning 'at times' fits better ...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,0.932277,0.421522
5,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,financiería,finanzas,error,vocabulary,The correct field of study here is 'finanzas' ...,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,financiería,finanzas,vocabulary,0.835563,0.587087
6,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,su,tu,error,pronoun,"Since the sentence is informal, it should use ...",2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,su,tu,pronoun,0.835563,0.521948
7,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,clubs,clubes,error,article,The correct plural form of 'club' in Spanish i...,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,clubs,clubes,article,0.835563,0.681720
8,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,entrar clubs,unirte a clubes,error,phraseFlow,The correct phrase is 'unirte a' meaning 'join'.,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,entrar clubs,unirte a clubes,phraseFlow,0.835563,0.496237
9,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,podría,podrías,error,pronoun,'Podrías' is needed to match the subject 'tú'.,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,podría,podrías,pronoun,0.835563,0.923415


In [114]:
#create a score where both the sentence and the mistake are involved

weight_mistake = 0.75
weight_sentence = 0.25

mistakes_df['difficulty_score'] = (
    (1 - mistakes_df['mistake_similarity']) * weight_mistake +
    (1 - mistakes_df['sentence_similarity']) * weight_sentence
)

mistakes_df.head(10)

,original_sentence,corrected_sentence,original_mistake,correction,mistake_type,category,explanation,timestamp,original_sentence_cleaned,corrected_sentence_cleaned,original_mistake_cleaned,correction_cleaned,category_standard,sentence_similarity,mistake_similarity,difficulty_score
0,Me gusta haciendo proyectos que me encantan. E...,Me gusta hacer proyectos que me encantan. Ento...,haciendo,hacer,error,None,The verb 'gustar' is followed by an infinitive...,2025-09-15 13:35:44.266,me gusta haciendo proyectos que me encantan. e...,me gusta hacer proyectos que me encantan. ento...,haciendo,hacer,None,0.985953,0.531973,0.354532
1,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,vocabulary,The word 'comentar' is more appropriate here a...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,0.932277,0.286470,0.552079
2,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,preposition,The correct preposition to use with 'comentar'...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,0.932277,0.685403,0.252879
3,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,vocabulary,"In this context, 'aportar' meaning 'contribute...",2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,0.932277,0.348006,0.505926
4,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,vocabulary,'En ocasiones' meaning 'at times' fits better ...,2025-09-15 13:43:26.877,ella puede hablar acerca de sus trabajos pasad...,ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,0.932277,0.421522,0.450789
5,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,financiería,finanzas,error,vocabulary,The correct field of study here is 'finanzas' ...,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,financiería,finanzas,vocabulary,0.835563,0.587087,0.350794
6,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,su,tu,error,pronoun,"Since the sentence is informal, it should use ...",2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,su,tu,pronoun,0.835563,0.521948,0.399648
7,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,clubs,clubes,error,article,The correct plural form of 'club' in Spanish i...,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,clubs,clubes,article,0.835563,0.681720,0.279819
8,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,entrar clubs,unirte a clubes,error,phraseFlow,The correct phrase is 'unirte a' meaning 'join'.,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,entrar clubs,unirte a clubes,phraseFlow,0.835563,0.496237,0.418931
9,Hay algunas clases acerca de financiería que p...,Hay algunas clases acerca de finanzas que podr...,podría,podrías,error,pronoun,'Podrías' is needed to match the subject 'tú'.,2025-09-15 13:48:52.932,hay algunas clases acerca de financiería que p...,hay algunas clases acerca de finanzas que podr...,podría,podrías,pronoun,0.835563,0.923415,0.098548


In [115]:
category_grouped = mistakes_df.groupby('category_standard')

category_summary = category_grouped.agg(
    count=('difficulty_score', 'count'),
    avg_difficulty=('difficulty_score', 'mean')
).reset_index()

#compute an emphasis score
category_summary['emphasis_score'] = category_summary['count'] * category_summary['avg_difficulty']

#normalize to 0-1
category_summary['emphasis_score'] /= category_summary['emphasis_score'].max()

category_summary.head(10)

,category_standard,count,avg_difficulty,emphasis_score
0,agreement,1,0.291216,0.072383
1,article,1,0.279819,0.069551
2,conjugation,8,0.248153,0.493438
3,genderNumberAgreement,2,0.109215,0.054292
4,idiomaticExpressions,1,0.034842,0.008660
5,phraseFlow,8,0.119913,0.238440
6,preposition,8,0.177840,0.353625
7,pronoun,3,0.330451,0.246406
8,punctuation,3,0.048810,0.036396
9,vocabulary,12,0.335271,1.000000
